In [ ]:
from langchain_core.runnables import (
    Runnable,
    RunnableLambda, 
    RunnablePassthrough, 
    RunnableParallel,
    RunnableConfig
)
from operator import itemgetter
from typing import Optional

### RunnableSquence

In [ ]:
def increment_x_by_one(x: int) -> int:
    return x+1

def fake_llm(x: int)->str:
    return f"Result = {x}"


In [ ]:
class MyFirstChain(Runnable[int, str]):
    def invoke(
        self, input:str, config: Optional[RunnableConfig] = None
    ) -> str:
        increment=increment_x_by_one(input)
        return fake_llm(increment)

In [ ]:
runnable = MyFirstChain()
runnable.invoke(1)

In [ ]:
chain1 = (
    increment_x_by_one 
    | RunnableLambda(fake_llm)
)


In [ ]:
for i in dir(chain1):
    print(i)

In [ ]:
print(chain1.__class__)
print(vars(chain1))

In [ ]:
print(chain1.InputType().__class__)
print(chain1.OutputType().__class__)

In [ ]:
result = chain1.invoke(1)
print(result)

In [ ]:
chain1 = (
    RunnableLambda(increment_x_by_one) 
    | fake_llm
)
result = chain1.invoke(1)
print(result)

In [ ]:
result = await chain1.ainvoke(1)
print(result)

In [ ]:
result = chain1.batch([1, 2, 3])
print(result)

In [ ]:
result = await chain1.abatch([1, 2, 3])
print(result)

### ParallelSequence

In [ ]:
def add_one(x: int) -> int:
    return x + 1

def mul_two(x: int) -> int:
    return x * 2

def mul_three(x: int) -> int:
    return x * 3

chain2 = RunnableLambda(add_one) | {  # this dict is coerced to a RunnableParallel
    "mul_two": mul_two,
    "mul_three": mul_three,
}

In [ ]:
print(chain2.InputType().__class__)
print(chain2.OutputType().__class__)
print(chain2.get_input_jsonschema())
print(chain2.get_output_jsonschema())

In [ ]:
result = chain2.invoke(1)
print(result.__class__)
print(result)

### RunnablePassThrough and itemgetter

In [ ]:
chain1 = (
    itemgetter("x") 
    | RunnableLambda(increment_x_by_one) 
    | fake_llm
)
result = chain1.invoke({"x": 1})
print(result)

In [ ]:
chain2 = RunnableParallel(
    origin=RunnablePassthrough(),
    output=increment_x_by_one
)

chain2.invoke(1)

### RunnableAssign

In [ ]:
chain3 = RunnablePassthrough().assign(
    y=itemgetter("x") | RunnableLambda(increment_x_by_one)
)

chain3.invoke({"x": 1})

In [24]:
chain4 = RunnablePassthrough().assign(
    y = lambda d: d.get("x", 0) + 1, 
    z = lambda d: d.get("x", 0) + 2
)

chain4.invoke({"x": 1})

{'x': 1, 'y': 2, 'z': 3}

In [25]:

chain5 = RunnablePassthrough().assign(
    k={
        "y": lambda d: d.get("x", 0) + 1, 
        "z": lambda d: d.get("x", 0) + 2
    }
)

chain5.invoke({"x": 1})

{'x': 1, 'k': {'y': 2, 'z': 3}}